In [1]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials
import os, time, uuid,io
from PIL import Image, ImageDraw, ImageFont
from matplotlib.pyplot import imshow
import requests,json
import base64
from openai import AzureOpenAI
import datetime
import fitz
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence.models import AnalyzeResult
from azure.ai.formrecognizer import DocumentAnalysisClient
import numpy as np
import pandas as pd
import re
from msg_parser import MsOxMessage

def find_last_digit(s): # 20240605
    for i in range (len(s)-1,-1,-1):
        if s[i].isdigit():
            return i
    return -1

def find_first_digit(s): # 20240605
    for i in range (len(s)):
        if s[i].isdigit():
            return i
    return -1
 
def get_money_string(s):
    #match_st = re.search(r'\d', s)  #查找第一个数字
    #start = match_st.start()  # 获取数字的开始位置
    #match_ed = re.search(r'\d+$', s) #查找最后一个数字,对于'0.00'输出是None
    #end = match_ed.start()  # 获取数字的开始位置
    start = find_first_digit(s)
    end = find_last_digit(s)
    #print(s)
    if start!=-1 and end !=-1 and start<=end:        
        #print("st:ed",start,end+1)
        return s[start:end+1] # 截取第一个非零数字及其后面的数字子串
    else:
        return s

In [2]:
# attachments classification
# <snippet_creds_auth> train model has been done from custom vision UI
def attach_classify(image_path):
    startTime_pdf2img = datetime.datetime.now() #记录开始时间
    PRED_ENDPOINT = "https://mmgcustomvision-prediction.cognitiveservices.azure.com/"
    prediction_key = "8a0a85e3b4844e6e91586c0d7a246cd4" #os.environ["VISION_PREDICTION_KEY"]
    prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
    predictor = CustomVisionPredictionClient(PRED_ENDPOINT, prediction_credentials)

    publish_iteration_name = "attach-classify-Iteration11" #用General A2做训练
    #publish_iteration_name = "attach-classify-Iteration10" #添加2个没有竖条的定额票做分类
    #publish_iteration_name = "attach-classify-Iteration9" #又添加2个发票类型，竖着小图和竖着歪着分类错的  0204宫向南和 0220邢照喆  20240605
    #publish_iteration_name = "attach-classify-Iteration7" #添加2个定额体检中文发票做分类,去掉了一些fffff和zzzzz的样本，效果不好
    #publish_iteration_name = "attach-classify-Iteration6" #添加2个定额体检中文发票做分类,去掉了一些fffff的样本,大多时候用的是这个
    #publish_iteration_name = "attach-classify-Iteration5"  #添加2个定额体检中文发票做分类
    #publish_iteration_name = "attach-classify-Iteration4"
    base_image_location = os.path.join (os.getcwd(), "attachments_images\\for-train-test\\test\\1-zzzzz")
    #print(base_image_location)
 
    project_id ="67008ec5-fd22-4e35-9a45-62622d5f62a6"   # Project Id from custom vision UI settings
    #判断图片大小，不能大于4M
    size_in_bytes = os.path.getsize(image_path)
    if size_in_bytes < 4*1024*1024 :
        with open(image_path, mode="rb")  as img_bytes:
                results = predictor.classify_image(project_id, publish_iteration_name, img_bytes) 
    else:
        image_data = Image.open(image_path)
        scale = 0.8
        new_width = int(image_data.width * (1 / scale))
        new_height = int(image_data.height * (1 / scale))
        resized_image = image_data.resize((new_width, new_height))
        img_io_bytes = io.BytesIO()
        resized_image = resized_image.convert("RGB")
        resized_image.save(img_io_bytes,format = "JPEG")
        img_bytes = img_io_bytes.getvalue()
        results = predictor.classify_image(project_id, publish_iteration_name, img_bytes)

    # Display the results.
    #for prediction in results.predictions:
    #print("\t" + results.predictions[0].tag_name + ": {0:.2f}%".format(results.predictions[0].probability * 100))
    # <snippet_test>
    #print(image_path, 'this attch classification done!')
    endTime_pdf2img = datetime.datetime.now() #结束时间
    #print('image classification time = ',(endTime_pdf2img-startTime_pdf2img).seconds)
    #img_bytes.close()
    return results.predictions[0]
    
def pdf2image(page,output_folder,file_name,i):   
    # 一页的pdf 转 png图像, jpg也是支持的，只需要把名字改成.jpg结尾就行
    startTime_pdf2img = datetime.datetime.now() #记录开始时间
    imagePath = output_folder + file_name + '_'+ str(i) + '.png'
    #print(pdfPath)
    rotate = int(0)
    #每个尺寸的缩放系数为1.3，这将为我们生成分辨率提高2.6倍的图像
    #此处若是不做设置，默认图片大小为：792*612，dpf=96
    zoom_x = 1.33333333 # (1.33333333 --> 1056*816) (2 --> 1584*1224)
    zoom_y = 1.33333333
    mat = fitz.Matrix(zoom_x,zoom_y).prerotate(rotate)
    pix = page.get_pixmap(matrix=mat,alpha=False)

    pix.save(imagePath)
    endTime_pdf2img = datetime.datetime.now() #结束时间
    #print('pdf2img时间=',(endTime_pdf2img-startTime_pdf2img).seconds)
    return imagePath

def invoice_keywords_judge(content, keywords):
    api_base = 'https://australiaeastaoaitry.openai.azure.com/' 
    api_key = '55fa016b5edc4fd986383b57156dc612'
    deployment_name = 'gpt-4-1106-review'
    api_version = '2024-02-01'

    client = AzureOpenAI(
        azure_endpoint=api_base,
        api_key=api_key,  
        api_version=api_version
    )
    #print(content)
    inputcontent = content + " \n 上面这段话中，有 " + keywords + " 这几个字么？有的话输出Yes，没有的话输出No"    
    #print(inputcontent)

    response = client.chat.completions.create(
        model=deployment_name,
        messages=[
            {
                "role": "user",
                "content": inputcontent,
            },
        ],
        temperature=0.7,
        top_p=0.95,
        max_tokens=2000
    )

    print(response.choices[0].message.content)  
    return response.choices[0].message.content

def approve_words_judge(content, keywords):
    api_base = 'https://australiaeastaoaitry.openai.azure.com/' 
    api_key = '55fa016b5edc4fd986383b57156dc612'
    deployment_name = 'gpt-4-1106-review'
    api_version = '2024-02-01'

    client = AzureOpenAI(
        azure_endpoint=api_base,
        api_key=api_key,  
        api_version=api_version
    )

    inputcontent = content + " \n In the paragraph above, does it contain the following words: " + keywords + " ？and at the meantime, does it contain a person who is the approver? If both answers are yes, output yes, elsewise output no."    
    #print(inputcontent)

    response = client.chat.completions.create(
        model=deployment_name,
        messages=[
            {
                "role": "user",
                "content": inputcontent,
            },
        ],
        temperature=0.7,
        top_p=0.95,
        max_tokens=2000
    )

    #print(response.choices[0].message.content)  
    return response.choices[0].message.content

def approve_GPT4vision_enhance_judge(image_path, keywords):
    # taxi receipt key words extraction by GPT4-vision with cv enhancement 
    startTime_pdf2img = datetime.datetime.now() #记录开始时间
    #print(image_path)
    encoded_image = base64.b64encode(open(image_path, 'rb').read()).decode('ascii')

    aoai_api_base = 'https://australiaeastaoaitry.openai.azure.com/' 
    gpt4v_deployment_name = 'gpt-4-vision'
    aoai_API_KEY = '55fa016b5edc4fd986383b57156dc612'

    base_url = f"{aoai_api_base}/openai/deployments/{gpt4v_deployment_name}" 
    headers = {   
        "Content-Type": "application/json",   
        "api-key": aoai_API_KEY 
    } 

    inputcontent = keywords + " If the answer is Yes, output yes, elsewise output no."
    # Prepare endpoint, headers, and request body 
    endpoint = f"{base_url}/extensions/chat/completions?api-version=2023-12-01-preview" 
    data = {
        "model": gpt4v_deployment_name,
        "enhancements": {
            "ocr": {
              "enabled": True
            },
            "grounding": {
              "enabled": True
            }
        },
        "dataSources": [
        {
            "type": "AzureComputerVision",
            "parameters": {
                "endpoint": "https://australiaeastcv.cognitiveservices.azure.com/",
                "key": "d3e20277b5574573a31902426e188084"
            }
        }],
        "messages": [       
            { "role": "user", 
              "content": [
                {
                    "type": "text",
                    "text": inputcontent
                },
                { 
                    "type": "image_url", 
                    "image_url": {"url": f"data:image/jpeg;base64,{encoded_image}"}
                }]
            } 
        ], 
        "temperature": 0.01,
        "top_p": 0.95,
        "max_tokens": 2000 
    } 
    
       # Make the API call   
    response = requests.post(endpoint, headers=headers, data=json.dumps(data))   

    endTime_pdf2img = datetime.datetime.now() #结束时间
    print('approve function GPT4-vision with cv enhancement post time=',(endTime_pdf2img-startTime_pdf2img).seconds)
    #print(f"Status Code: {response.status_code}")   
    result = response.json()
    #print(result['choices'][0]['message']['content'])
    return result['choices'][0]['message']['content']

def chi_invoice_extract_intell(file_path): # 输入的是这页的内容，输出的是提取出来的时间，类别，金额和币种
    #这里有个假设条件是，一页内容里已经包含了要找的一条信息的所有了。
    startTime_pdf2img = datetime.datetime.now() #记录开始时间
    endpoint = "https://docintelnew.cognitiveservices.azure.com/"
    key = "04045862a7494346a2e8016e5b7efdbc"
    #file_path = os.path.join (file_path, file)

    #model_id = "prebuilt-layout"
    #model_id = "prebuilt-invoice"
    #model_id = "Chi-Invoice" # customized model named as Chi-Invoice with 12 invoices and 7 key words
    model_id = "Chi-Invoice-20240524" # customized model named as Chi-Invoice with 14 invoices and 8 key words, include medical invoice
    #model_id = "Chi-Invoice-20240606" #增加侧着的2个，和倒着的1个样本
    document_analysis_client = DocumentIntelligenceClient(endpoint=endpoint, credential=AzureKeyCredential(key))
    with open(file_path, "rb") as f:
        poller = document_analysis_client.begin_analyze_document(
                model_id, analyze_request=f, content_type="application/octet-stream")
    result = poller.result()
    endTime_pdf2img = datetime.datetime.now() #结束时间
    #print('Document Intelligence new SDK use time=',(endTime_pdf2img-startTime_pdf2img).seconds)
    #print(result['documents'][0]['fields'])
    #print('发票号码 ', result['documents'][0]['fields']['invoice-number']['content'])
    #print('日期 ', result['documents'][0]['fields']['Data']['content'])
    #print('价税合计 ', result['documents'][0]['fields']['total-money']['content'])
    #print('销售方名称 ', result['documents'][0]['fields']['seller']['content'])
    #print('不计税金额 ', result['documents'][0]['fields']['pure-money']['content'])
    #print('税额 ', result['documents'][0]['fields']['tax-money']['content'])
    keywords = result['documents'][0]['fields']
    print(keywords)
    #print(result['content'])
    attach_date = attach_type = attach_amount = attach_currency = attach_yes_invoice = '' #下面的每一个都应该先判断一下存在，再赋值
    attach_date = keywords['Data']['content']
    if 'content' in keywords['total-money']:
        attach_amount = get_money_string(keywords['total-money']['content'])
    else:
        print('total-money is not detected!')
        pure_money = get_money_string(keywords['pure-money']['content'])
        tax_money = get_money_string(keywords['tax-money']['content'])
        attach_amount = str(float(pure_money) + float(tax_money))
    attach_currency = 'CNY' #定额增值税发票默认只有中国有，发票上都是以人民币币种结算的 
    if 'content' in keywords['service-content']:
        print('服务内容 ', keywords['service-content']['content'])
        if '住宿' in keywords['service-content']['content'] or '房屋租金' in keywords['service-content']['content']:
            attach_type = 'Accommodation(DOM)'
        elif '机票' in keywords['service-content']['content'] or '航' in keywords['service-content']['content']:
            attach_type = 'Airfares (DOM)'
        elif '运输' in keywords['service-content']['content']:
            attach_type = 'Taxi/CarHire/Incidentals (DOM)'
        elif '餐' in keywords['service-content']['content']:
            attach_type = 'Meals'
        elif '物流' in keywords['service-content']['content'] or '图书' in keywords['service-content']['content'] or '纸' in keywords['service-content']['content'] or '签证' in keywords['service-content']['content'] or '护照' in keywords['service-content']['content']: 
            attach_type = 'Office & Admin Supplies'
        elif '电信' in keywords['service-content']['content']:
            attach_type = 'Telephone Subsidies (Business)'
        elif '公证' in keywords['service-content']['content'] or '信息技术' in keywords['service-content']['content']:
            attach_type = 'Licences and Permits'
        elif '医疗' in keywords['service-content']['content'] or '药' in keywords['service-content']['content'] or '诊疗' in keywords['service-content']['content']:
            attach_type = 'Travel Health & Medicals'
    
    if 'content' in keywords['title']:
        print(keywords['title']['content'])
        if '普通' in keywords['title']['content']:
            attach_yes_invoice = 'Yes' 
        elif '专用' in keywords['title']['content']:
            keywords_res = invoice_keywords_judge(str(result['content']),"抵扣联") # 抵扣联三个字有可能识别的不准确，例如：扣识别成和
            if 'Yes' in keywords_res:
                attach_yes_invoice = 'No'
            else:
                attach_yes_invoice = 'Yes'
        else:
            print("type of invoice is missed!")
    f.close()
    return  attach_date, attach_type, attach_amount, attach_currency, attach_yes_invoice

def receipts_detection(image_path,file_name,output_folder): #需要发票是竖着可读的方向才行，横着那种不行
    # train and taxi receipts detection,including international meal receipts (save cut region and class)
    # <snippet_creds> train model has been done from custom vision UI
    PRED_ENDPOINT = "https://australiseastcustomvision-prediction.cognitiveservices.azure.com/"
    prediction_key = "a2ad6e09d1624fe08572a2ace482ee3e" #os.environ["VISION_PREDICTION_KEY"]
    prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
    predictor = CustomVisionPredictionClient(PRED_ENDPOINT, prediction_credentials)
    #publish_iteration_name = "taxi-train-Iteration4"
    #publish_iteration_name = "detection-Iteration7" #增加了两张高速费的样本做目标检测
    #publish_iteration_name = "detection-Iteration8" #增加了两张打车小票的样本做目标检测 0312温良
    publish_iteration_name = "detection-Iteration9" #增加了打车票和高速费的样本做目标检测1212章箐Oct 23 - Nov 22
    project_id ="8203d8be-dc10-482d-875d-a3e2520cb599"   # Project Id from custom vision UI settings
    print(image_path) 

    img = Image.open(image_path)
    width, height = img.size
    #判断图片大小，不能大于4M 20240606
    size_in_bytes = os.path.getsize(image_path)
    if size_in_bytes < 4*1024*1024 :
        with open(image_path, mode="rb")  as test_data:
                results = predictor.detect_image(project_id, publish_iteration_name, test_data)  
    else:
        scale = 0.8
        new_width = int(width * (1 / scale))
        new_height = int(height * (1 / scale))
        resized_image = img.resize((new_width, new_height))
        img_io_bytes = io.BytesIO()
        resized_image = resized_image.convert("RGB")
        resized_image.save(img_io_bytes,format = "JPEG")
        img_bytes = img_io_bytes.getvalue()
        results = predictor.detect_image(project_id, publish_iteration_name, img_bytes)
        
    # Save the results.    
    det_threshold = 0.6
    subimg_list = []
    if results is not None:
        index = 1
        for prediction in results.predictions:
            if prediction.probability > det_threshold:
                #print("\t" + prediction.tag_name + ": {0:.2f}% bbox.left = {1:.2f}, bbox.top = {2:.2f}, bbox.width = {3:.2f}, bbox.height = {4:.2f}".format(prediction.probability * 100, prediction.bounding_box.left, prediction.bounding_box.top, prediction.bounding_box.width, prediction.bounding_box.height))
                x = int(prediction.bounding_box.left*width+0.5)
                y = int(prediction.bounding_box.top*height+0.5)
                w = int(prediction.bounding_box.width*width+0.5)
                h = int(prediction.bounding_box.height*height+0.5)
                subimg = img.crop((x, y, x+w, y+h))
                subimg_path = output_folder + file_name + "_" + prediction.tag_name + "_" + str(index) + image_path[-4:]
                subimg_list.append([subimg_path,x,y,w,h])
                index = index + 1
                print(x,y,w,h)
                subimg.save(subimg_path)
    #test_data.close()
    print(file_name, 'sub image detection done!')
    return subimg_list

def keywords_extract_GPT4vision_enhance(image_path):
    # taxi receipt key words extraction by GPT4-vision with cv enhancement 
    startTime_pdf2img = datetime.datetime.now() #记录开始时间
    #print(image_path)
    encoded_image = base64.b64encode(open(image_path, 'rb').read()).decode('ascii')

    aoai_api_base = 'https://australiaeastaoaitry.openai.azure.com/' 
    gpt4v_deployment_name = 'gpt-4-vision'
    aoai_API_KEY = '55fa016b5edc4fd986383b57156dc612'

    base_url = f"{aoai_api_base}/openai/deployments/{gpt4v_deployment_name}" 
    headers = {   
        "Content-Type": "application/json",   
        "api-key": aoai_API_KEY 
    } 

    # Prepare endpoint, headers, and request body 
    endpoint = f"{base_url}/extensions/chat/completions?api-version=2023-12-01-preview" 
    data = {
        "model": gpt4v_deployment_name,
        "enhancements": {
            "ocr": {
              "enabled": True
            },
            "grounding": {
              "enabled": True
            }
        },
        "dataSources": [
        {
            "type": "AzureComputerVision",
            "parameters": {
                "endpoint": "https://australiaeastcv.cognitiveservices.azure.com/",
                "key": "d3e20277b5574573a31902426e188084"
            }
        }],
        "messages": [       
            { "role": "user", 
              "content": [
                {
                    "type": "text",
                    "text": "Find the date, currency and the total amount of money charged in the image, \
                    and determine its category from the following categories: \
                    meal, accommodation, airfares, taxi ticket, highway fees, train tickets, subscription fee, membership fee, medical, telephone, \
                    and output it in the format of 'Money': ;'Currency': ; 'Date': ;'Type': . \
                    Following the rules below:  \
                    1. Be based on facts, don't make up stories. \
                    2. If the output content can't been found, it should be outputed as blank. \
                    3. If it contains food, it should be identified as the meal type. \
                    4. If it contains mobile or wifi, it should be identified as telephone type. \
                    5. Determine the currency based on the content in the image. \
                    6. If it is a taxi ticket, the total amount of money charged is the total amount or Chinese called 实收金额, not the fare amount or Chinese called 金额. \
                    "
                    #4. Find the largest money and if it is larger than the total amount charged, the output Money should be the largest one.
                },
                { 
                    "type": "image_url", 
                    "image_url": {"url": f"data:image/jpeg;base64,{encoded_image}"}
                }]
            } 
        ], 
        "temperature": 0.01,
        "top_p": 0.95,
        "max_tokens": 2000 
    }   

    # Make the API call   
    response = requests.post(endpoint, headers=headers, data=json.dumps(data))   

    endTime_pdf2img = datetime.datetime.now() #结束时间
    print('GPT4-vision with cv enhancement post time=',(endTime_pdf2img-startTime_pdf2img).seconds)
    #print(f"Status Code: {response.status_code}")   
    result = response.json()
    print(result['choices'][0]['message']['content'])

    thislist = result['choices'][0]['message']['content'].split(';')
    print(thislist)
    thismoney = thislist[0]
    thiscurrency = thislist[1]
    thisdate = thislist[2] 
    thistype = thislist[3]
    #print(thismoney, thiscurrency, thisdate, thistype)
    attach_date = attach_type = attach_amount = attach_currency = ''
    if 'Currency' in thiscurrency:
        tmp,attach_currency = thiscurrency.split(':')
    domorint = '(DOM)'       
    if 'CNY' in attach_currency or 'yuan' in attach_currency or 'Yuan' in attach_currency or '元' in attach_currency or '¥' in attach_currency or 'RMB' in attach_currency:
        attach_currency = 'CNY'
        domorint = '(DOM)'
    else:
        domorint = '(INT)'
    
    if 'Type' in thistype:
        tmp,attach_type = thistype.split(':')
    if 'Taxi' in thistype or 'taxi' in thistype or 'train' in thistype or 'Train' in thistype or 'highway' in thistype or 'Highway' in thistype:
        attach_type = 'Taxi/CarHire/Incidentals' + domorint
    elif 'accommodation' in thistype or 'Accommodation' in thistype:
        attach_type = 'Accommodation' + domorint
    elif 'airfares' in thistype or 'Airfares' in thistype:
        attach_type = 'Airfares' + domorint
    elif 'Subscription' in thistype or 'subscription' in thistype:
        attach_type = 'Training-courses, Training, Seminar, Conferences'
    elif 'Membership' in thistype or 'membership' in thistype:
        attach_type = 'Employee Memberships/Subscript'
    elif 'Medical' in thistype or 'medical' in thistype:
        attach_type = 'Travel Health & Medicals'
    elif 'telephone' in thistype or 'Telephone' in thistype:
        attach_type = 'Telephone Subsidies (Business)'       
        
    if 'Date' in thisdate:
        pos = thisdate.index(':')+1 #因为Date里会混入小时分钟的表达里含有冒号，所以作此修改   20240605
        attach_date = thisdate[pos:]
        #tmp,attach_date = thisdate.split(':')
    if 'Money' in thismoney:
        tmp,attach_amount = thismoney.split(':')
        attach_amount = get_money_string(attach_amount)

    #print(attach_date,attach_type,attach_amount,attach_currency)
    return  attach_date,attach_type,attach_amount,attach_currency 

def get_image_area(image_path):
    image = Image.open(image_path)
    w,h = image.size	# 宽高像素
    imagePixmap = (float)(w*h/1000)
    #print("w,h:",w,h)
    #print("imagePixmap:", imagePixmap)
    return imagePixmap

In [7]:
attach_files_folder = 'C:\\Users\\pih\\pycodes\\MMG\\codes\\1221王晓\\'
temp_image_folder = './temp_image/'
file_type = "DOM"
#file_type = "DOMINT"
#file_type = "INT"
approve_path = ""
approve_flag = ""
mres = np.empty(shape=[0, 10], dtype=str) # 多加一个页数和一个页数上的item，也就是如果是pdf有多页,每页有多项的话，就会知道是第几页的第几项
for root, dirs, files in os.walk(attach_files_folder):
    for f in files:
        # only deal with pdf and image type attchment
        file_Path = os.path.join (root, f)
        print(file_Path)
        if f[-3:] == "jpg" or f[-3:] == "png": 
            cls = attach_classify(file_Path) 
            print(cls)
            if cls.tag_name == "zzzzz" and cls.probability > 0.45: #需要用score限制一下，因为没有分other类别，如果是一条table的邮件回复啥的，score会低。
                result = [f,'1','1',cls.tag_name,str(cls.probability),'','','','','']
                result[5],result[6],result[7],result[8],result[9] = chi_invoice_extract_intell(file_Path) 
                print(result)
                mres = np.append(mres, [result], axis = 0) 
            elif cls.tag_name == "sssss" and cls.probability > 0.45:
                subimg_list = receipts_detection(file_Path,f[:-4],temp_image_folder)
                if len(subimg_list) == 0:
                    print("image do not detect receipts!!! need to detect again!")
                    continue
                subimg_area = get_image_area(subimg_list[0][0]) 
                img_area = get_image_area(file_Path)
                print(subimg_area,0.7*img_area)
                if len(subimg_list) == 1 and file_type=="DOM" and subimg_area>0.7*img_area:
                    #国内文件类型错分成了小票类型? 哪个样例？
                    print("wrong image")
                    continue
                #开始提取每一个出租车小票的关键词信息
                j=1
                for subimg in subimg_list:
                    result = [f,'1',str(j),cls.tag_name,str(cls.probability),'','','','',''] 
                    #print(subimg)
                    result[5],result[6],result[7],result[8] = keywords_extract_GPT4vision_enhance(subimg[0])
                    j=j+1
                    print(result)
                    mres = np.append(mres, [result], axis = 0) 
            elif cls.tag_name == "wwwww" and cls.probability > 0.45:
                if file_type=="DOM": #一张出租车小票或者高速票被分成了文件类型，确认一下
                    #result[5],result[6],result[7],result[8] = keywords_extract_GPT4vision_enhance(file_Path)
                    print("DOM & wwwww & image")
                else:  #国内的报销的文档类型的附件忽略，只处理国外的
                    result = [f,'1','1',cls.tag_name,str(cls.probability),'','','','','']
                    result[5],result[6],result[7],result[8] = keywords_extract_GPT4vision_enhance(file_Path)
                    print(result)
                    if result[8] !='CNY':#国内外混杂情况下，如果这一份附件是国内的，忽略，只处理国外的。这里默认Uber的都是在国外打车，而且币种识别正确
                        mres = np.append(mres, [result], axis = 0) 
            #暂时先不处理fffff类型，这个更多和付费细节相关，不确定是不是和币种金额转换有关。  
        elif f[-3:] == "pdf":   
            pdfDoc = fitz.open(file_Path)
            for i in range(pdfDoc.page_count):
                image_path = pdf2image(pdfDoc[i],temp_image_folder,f[:-4],i+1) #对每一页做pdf到图像的转换,并保存
                print(image_path)
                cls = attach_classify(image_path)
                print(cls)
                if cls.tag_name == "zzzzz" and cls.probability > 0.45: #需要用score限制一下，因为没有分other类别，如果是一条table的邮件回复啥的，score会低。
                    result = [f,str(i+1),'1',cls.tag_name,str(cls.probability),'','','','','']
                    result[5],result[6],result[7],result[8],result[9] = chi_invoice_extract_intell(image_path) 
                    print(result)
                    mres = np.append(mres, [result], axis = 0) 
                elif cls.tag_name == "sssss" and cls.probability > 0.38: # 0118高辉晓-27-Expense Receipt_026的score分比较低，所以这里改成了0.38
                    save_name = f[:-4]+ '_page_' + str(i+1)
                    subimg_list = receipts_detection(image_path,save_name,temp_image_folder)
                    if len(subimg_list) == 0:
                        print("pdf do not detect receipts!!! need to detect again!")
                        continue
                    subimg_area = get_image_area(subimg_list[0][0]) 
                    img_area = get_image_area(image_path)
                    print(subimg_area,0.7*img_area)
                    if len(subimg_list) == 1 and file_type=="DOM" and subimg_area > 0.7*img_area:
                        #国内文件类型错分成了小票类型
                        print("wrong pdf")
                        continue
                    j=1
                    for subimg in subimg_list:
                        result = [f,str(i+1),str(j),cls.tag_name,str(cls.probability),'','','','','']
                        #print(subimg)
                        result[5],result[6],result[7],result[8] = keywords_extract_GPT4vision_enhance(subimg[0])
                        j=j+1
                        print(result)
                        mres = np.append(mres, [result], axis = 0) 
                elif cls.tag_name == "wwwww" and cls.probability > 0.45: 
                    #page_text = pdfDoc[i].get_text("text")
                    #print(page_text)
                    #if approve_flag =="":
                        #output_flag = approve_GPT4vision_enhance_judge(image_path,"Does it is contain the words: Authorithy to Travel?")
                        #print(output_flag)
                        #if 'yes' or 'Yes' in output_flag:
                        #    third_image_path = pdf2image(pdfDoc[i+2],temp_image_folder,f[:-4],i+2+1) #对每一页做pdf到图像的转换
                        #    third_output_flag = approve_GPT4vision_enhance_judge(third_image_path,"Does it is contain a manager approver? ")
                        #    print(third_output_flag)
                        #    if 'yes' or 'Yes' in third_output_flag:                 
                        #        approve_flag = "Yes"
                        #        approve_path = file_Path
                        #        break
                    if file_type!="DOM":#国内的报销的文档类型的附件忽略，只处理国外的
                        result = [f,str(i+1),'1',cls.tag_name,str(cls.probability),'','','','','']
                        result[5],result[6],result[7],result[8] = keywords_extract_GPT4vision_enhance(image_path)
                        print(result)
                        if result[8] =='' or result[8] ==' ':
                            break
                        if 'Taxi'in result[6] and result[8] !='CNY':#国内外混杂情况下，如果这一份附件是国内的，忽略，只处理国外的。这里默认Uber的都是在国外打车，而且币种识别正确
                            mres = np.append(mres, [result], axis = 0) 
                        if 'Medical' in result[6] or 'meal' in result[6]: #有中文水单做医疗体检的情况
                            mres = np.append(mres, [result], axis = 0) 
                        if 'Training' in result[6] or 'training' in result[6] or 'Subscript' in result[6] or 'Accommodation' in result[6]:
                            mres = np.append(mres, [result], axis = 0)
                #暂时先不处理fffff类型，这个更多和付费细节相关，不确定是不是和币种金额转换有关。
                elif cls.tag_name == "fffff" and cls.probability > 0.35: #0118高辉晓-12-Expense Receipt_011 的score分错了，还比较低，所以这里改成了0.35
                    #有可能是中文医院定额发票错分成fffff类型了
                    result = [f,str(i+1),'1',cls.tag_name,str(cls.probability),'','','','','']
                    result[5],result[6],result[7],result[8] = keywords_extract_GPT4vision_enhance(image_path)
                    if 'medical' in result[6] or 'Medical' in result[6]:
                        result[6] = 'Travel Health & Medicals'
                        mres = np.append(mres, [result], axis = 0)
                    if 'Taxi' in result[6] or 'Accommodation' in result[6] or 'Telephone' in result[6]:
                        mres = np.append(mres, [result], axis = 0)
            pdfDoc.close()
        elif f[-3:] == "msg": 
            msg_obj  = MsOxMessage(file_Path)
            msg_properties_dict = msg_obj.get_properties()
            #print(msg_properties_dict['Subject'])
            #print(msg_properties_dict['recipients'])
            #print(msg_properties_dict['Body'])
            output_flag = approve_words_judge(msg_properties_dict['Body'],"Authority to Travel request you submitted has been approved")
            print(output_flag)
            if 'yes' or 'Yes' in output_flag:
                approve_flag = "Yes"
                approve_path = file_Path


# 将numpy数组转换为pandas DataFrame
df = pd.DataFrame(mres, columns=['file_name', 'page_num','item_num','cls', 'score','Date','Type','Amount','Crcy','Accept_Invoice'])
print(df) 
print(approve_flag,approve_path)
out_csv_path = os.path.join(attach_files_folder,'attach_results.csv')
#df.to_csv(out_csv_path, encoding='utf_8_sig',index=False)
df.to_csv(out_csv_path, encoding='utf_8_sig')

C:\Users\pih\pycodes\MMG\codes\1221王晓\1221王晓-01-Expense Receipt_001.jpg
{'additional_properties': {}, 'probability': 0.9187108, 'tag_id': '7fbd22d7-3ef9-4c88-a2a5-e841c8df5203', 'tag_name': 'wwwww', 'bounding_box': None, 'tag_type': 'Regular'}
DOM & wwwww & image
C:\Users\pih\pycodes\MMG\codes\1221王晓\1221王晓-02-Expense Receipt_002.jpg
{'additional_properties': {}, 'probability': 0.9187108, 'tag_id': '7fbd22d7-3ef9-4c88-a2a5-e841c8df5203', 'tag_name': 'wwwww', 'bounding_box': None, 'tag_type': 'Regular'}
DOM & wwwww & image
C:\Users\pih\pycodes\MMG\codes\1221王晓\1221王晓-03-Expense Receipt_003.pdf
./temp_image/1221王晓-03-Expense Receipt_003_1.png
{'additional_properties': {}, 'probability': 0.849274, 'tag_id': 'a5578eee-69f9-41d8-bfb9-9e2d9f5651e4', 'tag_name': 'zzzzz', 'bounding_box': None, 'tag_type': 'Regular'}
{'invoice-number': {'type': 'string', 'valueString': ':23312000000130193540', 'content': ':23312000000130193540', 'boundingRegions': [{'pageNumber': 1, 'polygon': [633, 338, 768, 3

{'additional_properties': {}, 'probability': 0.9712749, 'tag_id': 'a5578eee-69f9-41d8-bfb9-9e2d9f5651e4', 'tag_name': 'zzzzz', 'bounding_box': None, 'tag_type': 'Regular'}
{'invoice-number': {'type': 'string', 'valueString': ':10882215', 'content': ':10882215', 'boundingRegions': [{'pageNumber': 1, 'polygon': [607, 46, 663, 45, 663, 57, 607, 57]}], 'confidence': 0.951, 'spans': [{'offset': 312, 'length': 9}]}, 'Data': {'type': 'string', 'valueString': ':2023 年 12 月 20 日', 'content': ':2023年12月20日', 'boundingRegions': [{'pageNumber': 1, 'polygon': [609, 67, 699, 67, 699, 82, 609, 82]}], 'confidence': 0.905, 'spans': [{'offset': 326, 'length': 12}]}, 'total-money': {'type': 'string', 'valueString': ')¥3186.00', 'content': ')¥3186.00', 'boundingRegions': [{'pageNumber': 1, 'polygon': [629, 370, 707, 371, 707, 386, 629, 385]}], 'confidence': 0.941, 'spans': [{'offset': 647, 'length': 9}]}, 'seller': {'type': 'string', 'valueString': '北 京 正 院 家 常 菜 有 限 公 司', 'content': '北京正院家常菜有限公司', 'bound

In [45]:
# taxi receipt key words extraction by only GPT4-vision withtout vision enhancement 
#
# Configuration
startTime_pdf2img = datetime.datetime.now() #记录开始时间
GPT4V_KEY = "55fa016b5edc4fd986383b57156dc612"
base_image_location = os.path.join (os.getcwd(), "temp_image")
test_image_name = "0118王庆佳-2-RRRRR_taxi_1.png"
IMAGE_PATH = os.path.join (base_image_location, test_image_name)
print(IMAGE_PATH)
encoded_image = base64.b64encode(open(IMAGE_PATH, 'rb').read()).decode('ascii')
headers = {
    "Content-Type": "application/json",
    "api-key": GPT4V_KEY,
}

# Payload for the request
payload = {
  "messages": [
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "Describe the image in detail."
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "image_url",
          "image_url": {"url": f"data:image/jpeg;base64,{encoded_image}"}
        }
      ]
    }
  ],
  "temperature": 0.7,
  "top_p": 0.95,
  "max_tokens": 800
}

GPT4V_ENDPOINT = "https://australiaeastaoaitry.openai.azure.com/openai/deployments/gpt-4-vision/chat/completions?api-version=2024-02-15-preview"

# Send request
try:
    response = requests.post(GPT4V_ENDPOINT, headers=headers, json=payload)
    response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
except requests.RequestException as e:
    raise SystemExit(f"Failed to make the request. Error: {e}")

endTime_pdf2img = datetime.datetime.now() #结束时间
print('GPT4-vision post time=',(endTime_pdf2img-startTime_pdf2img).seconds)
# Handle the response as needed (e.g., print or process)
print(response.text)

C:\Users\pih\pycodes\MMG\codes\pages\temp_image\0118王庆佳-2-RRRRR_taxi_1.png
GPT4-vision post time= 18
{"id":"chatcmpl-9KMqRGD6EkImZ6IOZURYomiRqHE3q","object":"chat.completion","created":1714641403,"model":"gpt-4","prompt_filter_results":[{"prompt_index":0,"content_filter_results":{"hate":{"filtered":false,"severity":"safe"},"self_harm":{"filtered":false,"severity":"safe"},"sexual":{"filtered":false,"severity":"safe"},"violence":{"filtered":false,"severity":"safe"}}}],"choices":[{"finish_reason":"stop","index":0,"message":{"role":"assistant","content":"The image is of a receipt from a taxi service. The receipt is printed on thermal paper with a mixture of red and blue ink. It is predominantly in Chinese characters, with some English words such as \"TAXI\" and \"RECEIPT.\"\n\nAt the top of the receipt, there are several strings of numbers, which appear to be codes or reference numbers. Below this, there is a red oval stamp that contains Chinese characters around the top and bottom, with t

In [65]:
#taxi receipt key words extraction by GPT4-vision with vision enhancement 有点慢
#
# Configuration
startTime_pdf2img = datetime.datetime.now() #记录开始时间
base_image_location = os.path.join (os.getcwd(), "temp_image")
test_image_name = "0118王庆佳-2-RRRRR_taxi_3.png"
IMAGE_PATH = os.path.join (base_image_location, test_image_name)
print(IMAGE_PATH)
encoded_image = base64.b64encode(open(IMAGE_PATH, 'rb').read()).decode('ascii')

aoai_api_base = 'https://australiaeastaoaitry.openai.azure.com/' 
gpt4v_deployment_name = 'gpt-4-vision'
aoai_API_KEY = '55fa016b5edc4fd986383b57156dc612'

base_url = f"{aoai_api_base}/openai/deployments/{gpt4v_deployment_name}" 
headers = {   
    "Content-Type": "application/json",   
    "api-key": aoai_API_KEY 
} 

# Prepare endpoint, headers, and request body 
endpoint = f"{base_url}/extensions/chat/completions?api-version=2023-12-01-preview" 
data = {
    "model": gpt4v_deployment_name,
    "enhancements": {
        "ocr": {
          "enabled": True
        },
        "grounding": {
          "enabled": True
        }
    },
    "dataSources": [
    {
        "type": "AzureComputerVision",
        "parameters": {
            "endpoint": "https://australiaeastcv.cognitiveservices.azure.com/",
            "key": "d3e20277b5574573a31902426e188084"
        }
    }],
    "messages": [ 
        { "role": "system", "content": "Find the currency, date and total amount of money in the image, and determine its category from the following categories: meal, accommodation, taxi ticket, highway fees, train tickets, and output it in the format of 'Money': ,'Currency': , 'Date': ,'Type': . Be based on facts, don't make up stories." }, 
        { "role": "user", 
        "content": [  
            { 
                "type": "image_url", 
                "image_url": {
                    "url": f"data:image/jpeg;base64,{encoded_image}"
                }
            }
        ]} 
    ], 
    "max_tokens": 2000 
}   

# Make the API call   
response = requests.post(endpoint, headers=headers, data=json.dumps(data))   

endTime_pdf2img = datetime.datetime.now() #结束时间
print('GPT4-vision with vision enhancement post time=',(endTime_pdf2img-startTime_pdf2img).seconds)
print(f"Status Code: {response.status_code}")   
#print(response.choices[0].message.content)
result = response.json()
print(result['choices'][0]['message']['content'])
thismoney, thiscurrency, thisdate, thistype = result['choices'][0]['message']['content'].split(',')
attach_date = attach_type = attach_amount = attach_currency = ''
if 'Taxi' in thistype:
    attach_type = 'Taxi/CarHire/Incidentals (INT)'
if 'Date' in thisdate:
    tmp,attach_date = thisdate.split(':')
if 'Money' in thismoney:
    tmp,attach_amount = thismoney.split(':')
if 'Currency' in thiscurrency:
    tmp,attach_currency = thiscurrency.split(':')
print(attach_date,attach_type,attach_amount,attach_currency)

C:\Users\pih\pycodes\MMG\codes\pages\temp_image\0118王庆佳-2-RRRRR_taxi_3.png
GPT4-vision with vision enhancement post time= 8
Status Code: 200
'Money': 57.00, 'Currency': CNY, 'Date': 2024-1-9, 'Type': Taxi Ticket.


TypeError: 'str' object is not callable